In [25]:
import numpy as np
import pandas as pd
import psycopg2
import os

In [22]:
conn = psycopg2.connect(
    host="wrds-pgdata.wharton.upenn.edu",
    port = 9737,
    database="wrds",
    user="rsigalov",
    password="9DL8GyKJ")

# Loading all companies to estimate
query = """
    select secid, cusip, issue_type
    from OPTIONM.SECURD
"""
query = query.replace('\n', ' ').replace('\t', ' ')
df_issue_type = pd.read_sql_query(query, conn)

om_cs_list = df_issue_type[
    (df_issue_type.issue_type == "0") | 
    (df_issue_type.issue_type == 0)]["secid"]

# 2. Use CRSP common share identifier
query = """
    select distinct
        permno
    from crsp.msenames
    where shrcd in (10,11) and exchcd in (1,2,3)
"""
query = query.replace('\n', ' ').replace('\t', ' ')
df_crsp_cs = pd.read_sql_query(query, conn)

# Getting all secids from the linking table that have
# a match in CRSP
om_crsp_link = pd.read_csv("data/linking_files/om_crsp_wrds_linking_table.csv")
om_crsp_link = om_crsp_link[om_crsp_link.score == 1]
crsp_cs_list = om_crsp_link[om_crsp_link["PERMNO"].isin(df_crsp_cs["permno"])]["secid"]

# Joining the two lists and dropping duplicates:
companies_to_estimate = list(om_cs_list) + list(crsp_cs_list)
companies_to_estimate = list(set(companies_to_estimate))

conn.close()

/Users/rsigalov/Dropbox/Projects/disaster-risk/DRenv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/rsigalov/Dropbox/Projects/disaster-risk/DRenv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [26]:
# Loading files with V/IV and checking their secids
def filterName(name, appendix_list):
    for appendix in appendix_list:
        if appendix in name:
            return True
    
    return False
    
def filterDf(df):
    return df['secid'].drop_duplicates()

appendix_list = ['_final_', '_march_2021_', '_missing_', '_new_release_']
file_list = os.listdir("data/output/")
file_list = [file for file in file_list if filterName(file, appendix_list)]
secid_loaded = pd.concat(
    [filterDf(pd.read_csv("data/output/" + file)) for file in file_list], ignore_index=True)

In [33]:
conn = psycopg2.connect(
    host="wrds-pgdata.wharton.upenn.edu",
    port = 9737,
    database="wrds",
    user="rsigalov",
    password="9DL8GyKJ")

query = "select distinct secid from OPTIONM.OPPRCD2021"
secid_df = pd.read_sql_query(query, conn)
conn.close()

/Users/rsigalov/Dropbox/Projects/disaster-risk/DRenv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [38]:
with open("account_data/wrds_user.txt") as f:
    wrds_username = f.readline()

with open("account_data/wrds_pass.txt") as f:
    wrds_password = f.readline()


In [50]:
import random

In [53]:
companies_to_estimate = list(secid_df[secid_df["secid"].isin(companies_to_estimate)]["secid"])

In [55]:
len(companies_to_estimate)

4215

In [57]:
# Splitting into 3 groups to estimate on 3 AWS machines:
companies_to_estimate_1 = []
companies_to_estimate_2 = []
companies_to_estimate_3 = []

for i_secid, secid in enumerate(companies_to_estimate):
    if i_secid % 3 == 0:
        companies_to_estimate_1.append(secid)
    elif i_secid % 3 == 1:
        companies_to_estimate_2.append(secid)
    else:
        companies_to_estimate_3.append(secid)

In [85]:
base_name = 'march_2022_update'
residual_companies = []
begin_year = 2021
end_year = 2021
delete_raw_option_files = True


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


## Removing duplicate observations

In [86]:
om_march_2021 = pd.concat(
    [pd.read_csv(f"data/output/{file}") for file in os.listdir("data/output") if "march_2021_update" in file],
    ignore_index=True)
om_new_release = pd.concat(
    [pd.read_csv(f"data/output/{file}") for file in os.listdir("data/output") if "new_release" in file],
    ignore_index=True)

In [107]:
# The duplicates by (secid, date, maturity) in the original files have the same entries, 
# so we can safely drop them. They likely occured because I loaded the same company
# multiple times in the scripts
om_march_2021 = om_march_2021.drop_duplicates()
om_new_release = om_new_release.drop_duplicates()

In [108]:
om_combined = pd.concat(
    [om_march_2021, om_new_release], ignore_index=True
)

In [111]:
cnt_combined = om_combined.groupby(["secid", "date", "T"])["secid"].count().rename("cnt").reset_index()
cnt_combined[cnt_combined["cnt"] > 1].sort_values("date")

,secid,date,T,cnt
3675456,125237.0,2019-01-02,0.021918,2
2004308,107939.0,2019-01-02,0.041096,2
2004307,107939.0,2019-01-02,0.021918,2
2001948,107935.0,2019-01-02,0.194521,2
2000460,107931.0,2019-01-02,0.539726,2
...,...,...,...,...
1640413,106674.0,2019-06-28,0.476712,2
1640414,106674.0,2019-06-28,0.553425,2
4659211,189767.0,2019-06-28,0.476712,2
1647128,106689.0,2019-06-28,0.304110,2


In [112]:
om_combined = om_combined.drop_duplicates()

In [113]:
om_combined

,secid,date,T,V_in_sample,IV_in_sample,V_clamp,IV_clamp,rn_prob_sigma,rn_prob_2sigma,rn_prob_20,rn_prob_40,rn_prob_60,rn_prob_80,V,IV,rn_prob_5,rn_prob_10,rn_prob_15
0,101519.0,2019-01-02,0.287671,0.075626,0.072310,0.084023,0.079023,0.146284,0.040933,0.076786,0.013611,0.004009,0.002198,NaN,NaN,NaN,NaN,NaN
1,101519.0,2019-01-03,0.284932,0.083289,0.081260,0.095371,0.090917,0.145614,0.037512,0.091777,0.016314,0.004411,0.002096,NaN,NaN,NaN,NaN,NaN
2,101519.0,2019-01-04,0.282192,0.083076,0.080071,0.093860,0.088587,0.142803,0.038609,0.086722,0.016137,0.004666,0.002373,NaN,NaN,NaN,NaN,NaN
3,101519.0,2019-01-04,0.534247,0.057431,0.056189,0.077988,0.074331,0.148046,0.036019,0.140206,0.032137,0.007947,0.002521,NaN,NaN,NaN,NaN,NaN
4,101519.0,2019-01-07,0.273973,0.083415,0.079805,0.096922,0.090335,0.126872,0.035536,0.073890,0.015380,0.005643,0.004843,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507594,189258.0,2019-06-20,0.249315,0.181624,0.174645,0.218922,0.204559,0.136033,0.032916,0.128998,0.030040,0.010250,0.008534,NaN,NaN,NaN,NaN,NaN
6507595,189258.0,2019-06-20,0.498630,0.156881,0.152643,0.196772,0.197413,0.108282,0.037155,0.135275,0.063862,0.025012,0.005659,NaN,NaN,NaN,NaN,NaN
6507596,189258.0,2019-06-24,0.238356,0.178703,0.175089,0.215046,0.204886,0.123432,0.027014,0.126668,0.028190,0.010326,0.010614,NaN,NaN,NaN,NaN,NaN
6507597,189258.0,2019-06-24,0.487671,0.128998,0.121243,0.153144,0.140414,0.167767,0.053833,0.182919,0.067303,0.017554,0.001935,NaN,NaN,NaN,NaN,NaN


## Looking at FINAL and MISSING file groups

In [127]:
def filterDF(df):
    return df[df["secid"] == 101540]

# Getting secids for all files in the "final" category
file_list = [file for file in os.listdir("data/output/") if "final" in file]
om_final = pd.concat([filterDF(pd.read_csv(f"data/output/{file}")) for file in file_list], ignore_index=True)

# Getting secids for all files in the "missing" category
file_list = [file for file in os.listdir("data/output/") if "missing" in file]
om_missing = pd.concat([filterDF(pd.read_csv(f"data/output/{file}")) for file in file_list], ignore_index=True)


In [92]:
# Counting duplicates observations
print(om_march_2021[["secid","date","T"]].shape)
print(om_march_2021[["secid","date","T"]].drop_duplicates().shape)
print(om_new_release[["secid","date","T"]].shape)
print(om_new_release[["secid","date","T"]].drop_duplicates().shape)
print(pd.concat([om_march_2021[['secid',"date","T"]], om_new_release[["secid","date","T"]]], 
    ignore_index=True).drop_duplicates().shape)

(3923811, 3)
(3909429, 3)
(2603405, 3)
(2598170, 3)
(5635630, 3)


In [94]:
cnt_march_2021 = om_march_2021.groupby(["secid","date","T"])["secid"].count().rename("cnt").reset_index()
cnt_new_release = om_new_release.groupby(["secid","date","T"])["secid"].count().rename("cnt").reset_index()

In [98]:
pd.merge(
    cnt_new_release[cnt_new_release["cnt"] > 1],
    om_march_2021, on=["secid", "date", "T"], how="left"
).sort_values(["secid","date","T"])

,secid,date,T,cnt,V_in_sample,IV_in_sample,V_clamp,IV_clamp,rn_prob_sigma,rn_prob_2sigma,rn_prob_20,rn_prob_40,rn_prob_60,rn_prob_80,V,IV,rn_prob_5,rn_prob_10,rn_prob_15
0,102276.0,2018-01-02,0.043836,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102276.0,2018-01-02,0.063014,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,102276.0,2018-01-02,0.293151,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,102276.0,2018-01-02,0.542466,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,102276.0,2018-01-03,0.079452,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230,213247.0,2019-06-27,0.306849,2,0.266409,0.271528,0.304704,0.300596,0.149862,0.003466,0.270399,0.067546,0.003819,3.219376e-06,NaN,NaN,NaN,NaN,NaN
5231,213247.0,2019-06-27,0.556164,2,0.245420,0.246194,0.305831,0.300804,0.140428,0.000740,0.345992,0.144889,0.028404,9.760338e-04,NaN,NaN,NaN,NaN,NaN
5232,213247.0,2019-06-28,0.131507,2,0.313150,0.308929,0.366420,0.363696,0.150425,0.009111,0.176306,0.017375,0.000223,8.800625e-08,NaN,NaN,NaN,NaN,NaN
5233,213247.0,2019-06-28,0.304110,2,0.265417,0.270429,0.302694,0.298882,0.148009,0.004279,0.266019,0.065798,0.004335,1.459112e-05,NaN,NaN,NaN,NaN,NaN


In [106]:
cnt_new_release[cnt_new_release["cnt"] > 1]["T"]
om_new_release[
    (om_new_release["secid"] == 102276) & 
    (om_new_release["date"] == "2018-01-02")
].sort_values(["T"])

,secid,date,T,V_in_sample,IV_in_sample,V_clamp,IV_clamp,rn_prob_sigma,rn_prob_2sigma,rn_prob_20,rn_prob_40,rn_prob_60,rn_prob_80
889789,102276.0,2018-01-02,0.043836,0.282109,0.279942,0.340891,0.335328,0.117341,0.023990,0.029371,0.000058,-0.002036,0.004067
2582374,102276.0,2018-01-02,0.043836,0.282109,0.279942,0.340891,0.335328,0.117341,0.023990,0.029371,0.000058,-0.002036,0.004067
889790,102276.0,2018-01-02,0.063014,0.253737,0.252379,0.318342,0.314439,0.121662,0.024025,0.045551,0.003010,-0.001051,0.004323
2582375,102276.0,2018-01-02,0.063014,0.253737,0.252379,0.318342,0.314439,0.121662,0.024025,0.045551,0.003010,-0.001051,0.004323
889791,102276.0,2018-01-02,0.293151,0.308386,0.313606,0.360154,0.353895,0.149896,0.003052,0.286235,0.082965,0.006839,0.000017
2582376,102276.0,2018-01-02,0.293151,0.308386,0.313606,0.360154,0.353895,0.149896,0.003052,0.286235,0.082965,0.006839,0.000017
889792,102276.0,2018-01-02,0.542466,0.238392,0.243398,0.310497,0.310543,0.129029,0.000015,0.358714,0.143933,0.020148,0.000092
2582377,102276.0,2018-01-02,0.542466,0.238392,0.243398,0.310497,0.310543,0.129029,0.000015,0.358714,0.143933,0.020148,0.000092


In [ ]:
cnt_new_release[cnt_new_release["cnt"] > 1]["T"]